In [1]:
import pandas as pd
import numpy as np
import gc
import catboost as cat
import lightgbm as lgbm

from sklearn.preprocessing import LabelEncoder

In [2]:
train = pd.read_csv(r'C:\Users\dlwks\OneDrive\바탕 화면\VSCode\DACON_CODE_REVIEW\제주도 도로\train.csv')
test = pd.read_csv(r'C:\Users\dlwks\OneDrive\바탕 화면\VSCode\DACON_CODE_REVIEW\제주도 도로\test.csv')

display(train.head())
display(test.head())

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,TRAIN_0000000,20220623,목,17,1,106,지방도1112호선,0,0,60.0,...,3,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0
1,TRAIN_0000001,20220728,목,21,2,103,일반국도11호선,0,0,60.0,...,0,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0
2,TRAIN_0000002,20211010,일,7,2,103,일반국도16호선,0,0,80.0,...,0,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0
3,TRAIN_0000003,20220311,금,13,2,107,태평로,0,0,50.0,...,0,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0
4,TRAIN_0000004,20211005,화,8,2,103,일반국도12호선,0,0,80.0,...,0,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0


,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,height_restricted,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted
0,TEST_000000,20220825,목,17,3,107,연삼로,0,0,70.0,...,0.0,0,산지2교,33.499427,126.541298,없음,제주은행사거리,33.500772,126.543837,있음
1,TEST_000001,20220809,화,12,2,103,일반국도12호선,0,0,70.0,...,0.0,3,중문입구,33.258507,126.427003,없음,관광단지입구,33.258119,126.415840,없음
2,TEST_000002,20220805,금,2,1,103,일반국도16호선,0,0,60.0,...,0.0,0,도순3교,33.258960,126.476508,없음,도순2교,33.259206,126.474687,없음
3,TEST_000003,20220818,목,23,3,103,일반국도11호선,0,0,70.0,...,0.0,0,아라주공아파트,33.473494,126.545647,없음,인다마을,33.471061,126.545467,없음
4,TEST_000004,20220810,수,17,3,106,번영로,0,0,70.0,...,0.0,0,부록교 시종점,33.501477,126.569223,없음,봉개교 시종점,33.496863,126.581230,없음


In [3]:
train.drop(columns = 'id', inplace = True)

In [4]:
train

,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,vehicle_restricted,...,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,20220623,목,17,1,106,지방도1112호선,0,0,60.0,0.0,...,3,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0
1,20220728,목,21,2,103,일반국도11호선,0,0,60.0,0.0,...,0,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0
2,20211010,일,7,2,103,일반국도16호선,0,0,80.0,0.0,...,0,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0
3,20220311,금,13,2,107,태평로,0,0,50.0,0.0,...,0,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0
4,20211005,화,8,2,103,일반국도12호선,0,0,80.0,0.0,...,0,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701212,20211104,목,16,1,107,-,0,0,50.0,0.0,...,0,대림사거리,33.422145,126.278125,없음,금덕해운,33.420955,126.273750,없음,20.0
4701213,20220331,목,2,2,107,-,0,0,80.0,0.0,...,3,광삼교,33.472505,126.424368,없음,광삼교,33.472525,126.424890,없음,65.0
4701214,20220613,월,22,2,103,일반국도12호선,0,0,60.0,0.0,...,0,고성교차로,33.447183,126.912579,없음,성산교차로,33.444121,126.912948,없음,30.0
4701215,20211020,수,2,2,103,일반국도95호선,0,0,80.0,0.0,...,0,제6광령교,33.443596,126.431817,없음,관광대학입구,33.444996,126.433332,없음,73.0


In [5]:
train['year'] = train['base_date'].apply(lambda x : str(x)[:4])
train['month'] = train['base_date'].apply(lambda x : str(x)[4:6])
train['day'] = train['base_date'].apply(lambda x : str(x)[6:])

test['year'] = test['base_date'].apply(lambda x : str(x)[:4])
test['month'] = test['base_date'].apply(lambda x : str(x)[4:6])
test['day'] = test['base_date'].apply(lambda x : str(x)[6:])

train['year'] = pd.to_numeric(train['year'])
train['month'] = pd.to_numeric(train['month'])
train['day'] = pd.to_numeric(train['day'])

test['year'] = pd.to_numeric(test['year'])
test['month'] = pd.to_numeric(test['month'])
test['day'] = pd.to_numeric(test['day'])

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4701217 entries, 0 to 4701216
Data columns (total 25 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   base_date              int64  
 1   day_of_week            object 
 2   base_hour              int64  
 3   lane_count             int64  
 4   road_rating            int64  
 5   road_name              object 
 6   multi_linked           int64  
 7   connect_code           int64  
 8   maximum_speed_limit    float64
 9   vehicle_restricted     float64
 10  weight_restricted      float64
 11  height_restricted      float64
 12  road_type              int64  
 13  start_node_name        object 
 14  start_latitude         float64
 15  start_longitude        float64
 16  start_turn_restricted  object 
 17  end_node_name          object 
 18  end_latitude           float64
 19  end_longitude          float64
 20  end_turn_restricted    object 
 21  target                 float64
 22  year              

In [7]:
train['month_sin'] = np.sin(2 * np.pi * train['month'] / 12)
train['month_cos'] = np.cos(2 * np.pi * train['month'] / 12)
train['day_sin'] = np.sin(2 * np.pi * train['day'] / 31)
train['day_cos'] = np.cos(2 * np.pi * train['day'] / 31)
train['hour_sin'] = np.sin(2 * np.pi * train['base_hour'] / 24)
train['hour_cos'] = np.cos(2 * np.pi * train['base_hour'] / 24)
train['spring'] = train['month'].apply(lambda x : 1 if x in [3, 4, 5] else 0)
train['summer'] = train['month'].apply(lambda x : 1 if x in [6, 7, 8] else 0)
train['fall'] = train['month'].apply(lambda x : 1 if x in [9, 10, 11] else 0)
train['winter'] = train['month'].apply(lambda x : 1 if x in [12, 1, 2] else 0)
train['spring_sin'] = np.sin(2 * np.pi * train['spring'] / 3)
train['spring_cos'] = np.cos(2 * np.pi * train['spring'] / 3)
train['summer_sin'] = np.sin(2 * np.pi * train['summer'] / 3)
train['summer_cos'] = np.cos(2 * np.pi * train['summer'] / 3)
train['fall_sin'] = np.sin(2 * np.pi * train['fall'] / 3)
train['fall_cos'] = np.cos(2 * np.pi * train['fall'] / 3)
train['winter_sin'] = np.sin(2 * np.pi * train['winter'] / 3)
train['winter_cos'] = np.cos(2 * np.pi * train['winter'] / 3)

test['month_sin'] = np.sin(2 * np.pi * test['month'] / 12)
test['month_cos'] = np.cos(2 * np.pi * test['month'] / 12)
test['day_sin'] = np.sin(2 * np.pi * test['day'] / 31)
test['day_cos'] = np.cos(2 * np.pi * test['day'] / 31)
test['hour_sin'] = np.sin(2 * np.pi * test['base_hour'] / 24)
test['hour_cos'] = np.cos(2 * np.pi * test['base_hour'] / 24)
test['spring'] = test['month'].apply(lambda x : 1 if x in [3, 4, 5] else 0)
test['summer'] = test['month'].apply(lambda x : 1 if x in [6, 7, 8] else 0)
test['fall'] = test['month'].apply(lambda x : 1 if x in [9, 10, 11] else 0)
test['winter'] = test['month'].apply(lambda x : 1 if x in [12, 1, 2] else 0)
test['spring_sin'] = np.sin(2 * np.pi * test['spring'] / 3)
test['spring_cos'] = np.cos(2 * np.pi * test['spring'] / 3)
test['summer_sin'] = np.sin(2 * np.pi * test['summer'] / 3)
test['summer_cos'] = np.cos(2 * np.pi * test['summer'] / 3)
test['fall_sin'] = np.sin(2 * np.pi * test['fall'] / 3)
test['fall_cos'] = np.cos(2 * np.pi * test['fall'] / 3)
test['winter_sin'] = np.sin(2 * np.pi * test['winter'] / 3)
test['winter_cos'] = np.cos(2 * np.pi * test['winter'] / 3)

train

,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,vehicle_restricted,...,fall,winter,spring_sin,spring_cos,summer_sin,summer_cos,fall_sin,fall_cos,winter_sin,winter_cos
0,20220623,목,17,1,106,지방도1112호선,0,0,60.0,0.0,...,0,0,0.000000,1.0,0.866025,-0.5,0.000000,1.0,0.0,1.0
1,20220728,목,21,2,103,일반국도11호선,0,0,60.0,0.0,...,0,0,0.000000,1.0,0.866025,-0.5,0.000000,1.0,0.0,1.0
2,20211010,일,7,2,103,일반국도16호선,0,0,80.0,0.0,...,1,0,0.000000,1.0,0.000000,1.0,0.866025,-0.5,0.0,1.0
3,20220311,금,13,2,107,태평로,0,0,50.0,0.0,...,0,0,0.866025,-0.5,0.000000,1.0,0.000000,1.0,0.0,1.0
4,20211005,화,8,2,103,일반국도12호선,0,0,80.0,0.0,...,1,0,0.000000,1.0,0.000000,1.0,0.866025,-0.5,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701212,20211104,목,16,1,107,-,0,0,50.0,0.0,...,1,0,0.000000,1.0,0.000000,1.0,0.866025,-0.5,0.0,1.0
4701213,20220331,목,2,2,107,-,0,0,80.0,0.0,...,0,0,0.866025,-0.5,0.000000,1.0,0.000000,1.0,0.0,1.0
4701214,20220613,월,22,2,103,일반국도12호선,0,0,60.0,0.0,...,0,0,0.000000,1.0,0.866025,-0.5,0.000000,1.0,0.0,1.0
4701215,20211020,수,2,2,103,일반국도95호선,0,0,80.0,0.0,...,1,0,0.000000,1.0,0.000000,1.0,0.866025,-0.5,0.0,1.0


In [8]:
train['day_of_week'].value_counts()

day_of_week
금    684024
수    675583
목    674070
일    673632
토    669767
화    662498
월    661643
Name: count, dtype: int64

In [9]:
def map_day_of_week(day):
    days = {'월' : 0, '화' : 1, '수' : 2, '목' : 3, '금' : 4, '토' : 5, '일' : 6}
    return days.get(day)

train['day_of_week'] = train['day_of_week'].map(map_day_of_week)
test['day_of_week'] = test['day_of_week'].map(map_day_of_week)

train

,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,vehicle_restricted,...,fall,winter,spring_sin,spring_cos,summer_sin,summer_cos,fall_sin,fall_cos,winter_sin,winter_cos
0,20220623,3,17,1,106,지방도1112호선,0,0,60.0,0.0,...,0,0,0.000000,1.0,0.866025,-0.5,0.000000,1.0,0.0,1.0
1,20220728,3,21,2,103,일반국도11호선,0,0,60.0,0.0,...,0,0,0.000000,1.0,0.866025,-0.5,0.000000,1.0,0.0,1.0
2,20211010,6,7,2,103,일반국도16호선,0,0,80.0,0.0,...,1,0,0.000000,1.0,0.000000,1.0,0.866025,-0.5,0.0,1.0
3,20220311,4,13,2,107,태평로,0,0,50.0,0.0,...,0,0,0.866025,-0.5,0.000000,1.0,0.000000,1.0,0.0,1.0
4,20211005,1,8,2,103,일반국도12호선,0,0,80.0,0.0,...,1,0,0.000000,1.0,0.000000,1.0,0.866025,-0.5,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701212,20211104,3,16,1,107,-,0,0,50.0,0.0,...,1,0,0.000000,1.0,0.000000,1.0,0.866025,-0.5,0.0,1.0
4701213,20220331,3,2,2,107,-,0,0,80.0,0.0,...,0,0,0.866025,-0.5,0.000000,1.0,0.000000,1.0,0.0,1.0
4701214,20220613,0,22,2,103,일반국도12호선,0,0,60.0,0.0,...,0,0,0.000000,1.0,0.866025,-0.5,0.000000,1.0,0.0,1.0
4701215,20211020,2,2,2,103,일반국도95호선,0,0,80.0,0.0,...,1,0,0.000000,1.0,0.000000,1.0,0.866025,-0.5,0.0,1.0


In [10]:
train.isnull().sum()

base_date                0
day_of_week              0
base_hour                0
lane_count               0
road_rating              0
road_name                0
multi_linked             0
connect_code             0
maximum_speed_limit      0
vehicle_restricted       0
weight_restricted        0
height_restricted        0
road_type                0
start_node_name          0
start_latitude           0
start_longitude          0
start_turn_restricted    0
end_node_name            0
end_latitude             0
end_longitude            0
end_turn_restricted      0
target                   0
year                     0
month                    0
day                      0
month_sin                0
month_cos                0
day_sin                  0
day_cos                  0
hour_sin                 0
hour_cos                 0
spring                   0
summer                   0
fall                     0
winter                   0
spring_sin               0
spring_cos               0
s

In [11]:
str_col = ['day_of_week','start_turn_restricted','end_turn_restricted']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

In [12]:
y_train = train['target']

X_train = train.drop(['base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

test = test.drop(['id','base_date', 'road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

In [13]:
print(X_train.shape)
print(y_train.shape)
print(test.shape)

(4701217, 37)
(4701217,)
(291241, 37)


In [14]:
from sklearn.model_selection import train_test_split

X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print("X_train_split shape:", X_train_split.shape)
print("X_test_split shape:", X_test_split.shape)
print("y_train_split shape:", y_train_split.shape)
print("y_test_split shape:", y_test_split.shape)

X_train_split shape: (3760973, 37)
X_test_split shape: (940244, 37)
y_train_split shape: (3760973,)
y_test_split shape: (940244,)


In [15]:
lgbm = lgbm.LGBMRegressor(random_state = 42)

lgbm.fit(X_train_split, y_train_split)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.199466 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1249
[LightGBM] [Info] Number of data points in the train set: 3760973, number of used features: 36
[LightGBM] [Info] Start training from score 42.786869


LGBMRegressor(random_state=42)

In [16]:
y_pred = lgbm.predict(X_test_split)

In [17]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test_split, y_pred)
print("MAE :", mae)

MAE : 4.821843463844398
